# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [36]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [37]:
# 2) seu código aqui 
# Conte o número de valores missing por variável.
sinasc.isna().sum().sort_values(ascending=False)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DTCADASTRO        0
NUMEROLOTE        0
VERSAOSIST        0
ORIGEM            0
Length: 69, dtype: int64

In [ ]:
# 2.1)
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
missing_values_table(sinasc)

In [38]:
# 3) seu código aqui - Então crie uma seleção dessa base somente com as colunas que interessam. São elas: ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',  'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
# Refaça a contagem de valores missings.
df=sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',  'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df.isna().sum().sort_values(ascending=False)


QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [39]:
# 4) seu código aqui
# Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.
print ("A coluna APGAR5 possui " + str(len(df)) + " colunas, " + ("sendo " + str(df.APGAR5.isnull().sum()) + " missing.\n")   )   
df['APGAR5'].dropna()


A coluna APGAR5 possui 27028 colunas, sendo 103 missing.



0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64

In [40]:
# 5) seu código aqui
# observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9.
sinasc[['ESTCIVMAE', 'CONSULTAS']].isna().sum()
est = sinasc[['ESTCIVMAE', 'CONSULTAS']].fillna(9)
est.sort_values(by='ESTCIVMAE',ascending=False)



,ESTCIVMAE,CONSULTAS
18090,9.0,3
18128,9.0,4
17875,9.0,4
9408,9.0,4
9411,9.0,4
...,...,...
16063,1.0,4
2963,1.0,3
2962,1.0,2
16071,1.0,4


In [41]:
# 6) Seu código aqui
# Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
df['QTDFILVIVO'].fillna(0,inplace = True)
df['QTDFILVIVO'].sort_values()

C:\Users\Allan\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


0         0.0
9104      0.0
9105      0.0
19813     0.0
19812     0.0
         ... 
1854     12.0
22442    12.0
6831     12.0
16018    14.0
11365    30.0
Name: QTDFILVIVO, Length: 27028, dtype: float64

In [42]:
# 7) seu código aqui
# Das restantes, decida que valores te parecem mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

#LOCNASC - Local de ocorrência do nascimento, conforme a tabela: 9: Ignorado
#IDADEMAE - Idade da mãe em anos
#ESCMAE - Escolaridade, Anos de estudo concluídos: 9: Ignorado
#CONSULTAS - Número de consultas de pré-natal: 9: Ignorado
#QTDFILVIVO - Número de filhos vivos

#colunas contendo valores missing:
#APGAR5 - Apgar no quinto minuto 00 a 10         
#GESTACAO - Semanas de gestação, conforme a tabela: 9: Ignorado
#ESTCIVMAE - Estado civil, conforme a tabela: 9: Ignorado     
#ESCMAE - Escolaridade, anos de estudo concluídos: 9: Ignorado         
#GRAVIDEZ - Tipo de gravidez, conforme a tabela: 9: Ignorado

campo = 'GRAVIDEZ'
print(round((df[[campo]].isnull().sum() / len(df)) * 100,2))
df[campo].fillna('Ignorado',inplace = True)
uniq = df[[campo]].drop_duplicates()
uniq.sort_values(by=campo)
#JUSTIFICATIVA: A estrutura de dados comporta o valor 9: Ignorado



GRAVIDEZ    0.29
dtype: float64


,GRAVIDEZ
23,Dupla
401,Ignorado
8188,Tríplice e mais
0,Única


In [43]:
campo = 'ESCMAE'
print(f'Percentual de missing {round((df[[campo]].isnull().sum() / len(df)) * 100,2)} \n')
print(f'Total de missing {df[[campo]].isnull().sum()}\n')
df[campo].fillna(method='ffill', inplace = True)
df[campo].dropna()
uniq = df[[campo]].drop_duplicates()
uniq.sort_values(by=campo)
#JUSTIFICATIVA:  Os dados faltantes representaram 1,15% do total, justificando o preenchimento com o dado mais próximo por representar baixo impacto na analálise.

Percentual de missing ESCMAE    1.15
dtype: float64 

Total de missing ESCMAE    312
dtype: int64



C:\Users\Allan\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,ESCMAE
5,1 a 3 anos
3,12 anos ou mais
11,4 a 7 anos
0,8 a 11 anos
1024,Nenhuma


In [44]:
campo = 'ESTCIVMAE'
print(f'Percentual de missing {round((df[[campo]].isnull().sum() / len(df)) * 100,2)} \n')
print(f'Total de missing {df[[campo]].isnull().sum()}\n')
df[campo].fillna(9,inplace = True)
df[campo].dropna()
uniq = df[[campo]].drop_duplicates()
uniq.sort_values(by=campo)
#JUSTIFICATIVA: A estrutura de dados comporta o valor 9: Ignorado

Percentual de missing ESTCIVMAE    1.17
dtype: float64 

Total de missing ESTCIVMAE    317
dtype: int64



C:\Users\Allan\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,ESTCIVMAE
6,1.0
1,2.0
197,3.0
40,4.0
0,5.0
2,9.0


In [45]:
campo = 'GESTACAO'
print(f'Percentual de missing {round((df[[campo]].isnull().sum() / len(df)) * 100,2)} \n')
print(f'Total de missing {df[[campo]].isnull().sum()}\n')
print(f'OCORRÊNCIAS DE {df[[campo]].value_counts()}\n')

df[campo].fillna('Ignorado',inplace = True)
df[campo].dropna()
uniq = df[[campo]].drop_duplicates()
uniq.sort_values(by=campo)
#JUSTIFICATIVA: A estrutura de dados comporta o valor 9: Ignorado

Percentual de missing GESTACAO    4.56
dtype: float64 

Total de missing GESTACAO    1232
dtype: int64

OCORRÊNCIAS DE GESTACAO           
37 a 41 semanas        22589
32 a 36 semanas         2249
42 semanas e mais        700
28 a 31 semanas          165
22 a 27 semanas           86
Menos de 22 semanas        7
dtype: int64



C:\Users\Allan\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,GESTACAO
182,22 a 27 semanas
198,28 a 31 semanas
20,32 a 36 semanas
0,37 a 41 semanas
16,42 semanas e mais
1695,Ignorado
10980,Menos de 22 semanas


In [46]:
# 8) seu código aqui
# O Apgar possui uma classificação indicando se o bebê passou por asfixia:
    #Entre 8 e 10 está em uma faixa 'normal'.
    #Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
    #Entre 4 e 5 significa 'asfixia moderada'.
    #Entre 0 e 3 significa 'asfixia severa'.
#Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.


sinasc.loc[sinasc['APGAR5'] <= 3, 'CLASS_APGAR'] = 'asfixia severa'
sinasc.loc[(sinasc['APGAR5'] > 3) & (sinasc['APGAR5'] <= 5),
       'CLASS_APGAR'] = 'asfixia moderada'
sinasc.loc[(sinasc['APGAR5'] > 5) & (sinasc['APGAR5'] <= 7),
       'CLASS_APGAR'] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR5'] > 7), 'CLASS_APGAR'] = 'normal'
sinasc[['CLASS_APGAR']].value_counts()

CLASS_APGAR     
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
dtype: int64

In [47]:
# 9) seu código aqui
# Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 
df.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao', 'gravidez', 'consultas', 'apgar_5']
df.columns

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar_5'],
      dtype='object')